In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile as tiff 
from skimage.transform import resize

BASE_PATH = "../input/hubmap-organ-segmentation/"

### **MetaData Load**

In [ ]:
train = pd.read_csv('../input/hubmap-organ-segmentation/train.csv')
test = pd.read_csv('../input/hubmap-organ-segmentation/test.csv')

In [ ]:
train

### **Masking and Unmasking Functions**

In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 
def rle2mask(mask_rle, shape=(1600,256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [ ]:
img_id_1 = 9777
img_1 = tiff.imread(BASE_PATH + "train_images/" + str(img_id_1) + ".tiff")
print(img_1.shape)

img_id_2 = 9904
img_2 = tiff.imread(BASE_PATH + "train_images/" + str(img_id_2) + ".tiff")
print(img_2.shape)

**Getting the Mask Image**

In [ ]:
mask_1 = rle2mask(train[train["id"]==9777]["rle"].iloc[-1], (img_1.shape[1], img_1.shape[0]))
print(mask_1.shape)

mask_2 = rle2mask(train[train["id"]==9904]["rle"].iloc[-1], (img_2.shape[1], img_2.shape[0]))
print(mask_2.shape)

#### **Orginal Image Size Mask with Acutal Image**

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_1)
plt.imshow(mask_1, cmap='coolwarm', alpha=0.5)

plt.figure(figsize=(10,10))
plt.imshow(img_2)
plt.imshow(mask_2, cmap='coolwarm', alpha=0.5)

### **Resizing Both Mask Image and Actual Image**

In [ ]:
print('Image 1 Resizing.......')
mask_1 = resize(mask_1, (mask_1.shape[1] // 7, mask_1.shape[0] // 7))
print(mask_1.shape)

img_1 = resize(img_1, (img_1.shape[1] // 7, img_1.shape[0] // 7))
print(img_1.shape)

print('\nImage 2 Resizing.......')
mask_2 = resize(mask_2, (mask_2.shape[1] // 7, mask_2.shape[0] // 7))
print(mask_1.shape)

img_2 = resize(img_2, (img_2.shape[1] // 7, img_2.shape[0] // 7))
print(img_2.shape)

#### **Resized Mask Image with Actual Image**

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_1)
plt.imshow(mask_1, cmap='coolwarm', alpha=0.5)

plt.figure(figsize=(10,10))
plt.imshow(img_2)
plt.imshow(mask_2, cmap='coolwarm', alpha=0.5)

# 🥳 If You find this kernal useful Upvote 🥳🎉👏